In [0]:
#### GOOGLE DRIVE SPECIFIC ##########################
# Make sure that you have GPU selected in the Runtime
# If you do, will print Found GPU at: /device:GPU:0
# Else go to Runtime -> Change Runtime Type

import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [2]:
# CODE SNIPPET TO ACCESS THE FILES IN GOOGLE DRIVE (GO TO BROWSER AND VERIFY)
# THEN YOU CAN ACCESS THE FILES ON LEFT SIDEBAR (copy path)
# (https://colab.research.google.com/drive/1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q#scrollTo=H4SJ-tGNkOeY)

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
# drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
# !ls "/content/drive/My Drive"


Mounted at /content/drive


In [0]:
# googlepath exists for googledrive finding the files. 
googlepath = "drive/My Drive/SeniorDesign19"
foldernamepath = googlepath + "/Data/32BY32"
savedpath = googlepath + "/TrafficSignGAN/savedImages/"
modelpath = googlepath + "/TrafficSignGAN/savedModels/"

In [0]:
## Some Parameters to run code

# Make this True when you want to output single images. Keep it at false
# when you want to train the model. Note in either case make sure that
# only the models you want to use for generating or continuing training on
# are the only ones in the folder savedModels.

# Approach: First keep this false, train model, then make true and output
# single images.
runOnlyOneModel = True
                       

# *Note: Be sure to remove the savedModels when training
# a new class else it will start training using that.
classToTrain = 11

### For reference in our dataset: Largest 10 Classes ###
##Done
# # Class 28
# # Class 54
# # Class 3
# # Class 5
# # Class 55
# # Class 35
# # Class 7
# # Class 30
# # Class 16
# # Class 11

## To Do



In [5]:
from __future__ import print_function, division

from keras.layers import Input, Dense, Flatten, Dropout, Reshape
from keras.layers import BatchNormalization, Activation, Conv2D, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model, load_model
from keras.optimizers import Adam

from keras.datasets import cifar10
import keras.backend as K

from keras.models import model_from_json


import matplotlib.pyplot as plt

import sys
import numpy as np
import cv2
import os
import re

import numpy as np

import random


Using TensorFlow backend.


In [0]:
def get_generator(input_layer):
  '''
  Requires the input layer as input, outputs the model and the final layer
  '''
  
  hid = Dense(128 * 16 * 16, activation='relu')(input_layer)    
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)
  hid = Reshape((16, 16, 128))(hid)

  hid = Conv2D(128, kernel_size=5, strides=1,padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)    
  #hid = Dropout(0.5)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2DTranspose(128, 4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=5, strides=1, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  #hid = Dropout(0.5)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=5, strides=1, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)
                      
  hid = Conv2D(3, kernel_size=5, strides=1, padding="same")(hid)
  out = Activation("tanh")(hid)

  model = Model(input_layer, out)
  model.summary()
  
  return model, out

In [0]:
def get_discriminator(input_layer):
  '''
  Requires the input layer as input, outputs the model and the final layer
  '''

  hid = Conv2D(128, kernel_size=3, strides=1, padding='same')(input_layer)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Flatten()(hid)
  hid = Dropout(0.4)(hid)
  out = Dense(1, activation='sigmoid')(hid)

  model = Model(input_layer, out)

  model.summary()

  return model, out

In [0]:
from keras.preprocessing import image

def generate_noise(n_samples, noise_dim):
  X = np.random.normal(0, 1, size=(n_samples, noise_dim))
  return X

def show_imgs(batchidx):
  noise = generate_noise(9, 100)
  gen_imgs = generator.predict(noise)
  
  if runOnlyOneModel:
    # want to just output single images
    fig, axs = plt.subplots(1, 1)
    count = 0
    for i in range(9):
      # Dont scale the images back, let keras handle it
      img = image.array_to_img(gen_imgs[count], scale=True)
      axs.imshow(img)
      axs.axis('off')
      count += 1
      fig.savefig(savedpath + "class_{}_singleImages/".format(classToTrain) + "{}.png".format(batchidx + "_generated_{}".format(str(count))))
      plt.close()
  else:
    fig, axs = plt.subplots(3, 3)
    count = 0
    for i in range(3):
      for j in range(3):
        # Dont scale the images back, let keras handle it
        img = image.array_to_img(gen_imgs[count], scale=True)
        axs[i,j].imshow(img)
        axs[i,j].axis('off')
        count += 1
    #   plt.show()
    fig.savefig(savedpath + "class_{}_trainingProgression/".format(classToTrain) + "{}.png".format(batchidx))
    plt.close()

In [9]:
# GAN creation
starting_epoch = 0

model_names = os.listdir(modelpath)
model_names.sort(key = lambda x : (int(x.split('_')[2]), x.split('_')[3][:4]))
# print(model_names)

if len(model_names) != 0:
    # disc, gan, gen 
    one = model_names[-3]
    m = re.findall(r'[a-z]+', one)
    assert m[1] == "disc", print("wanted: disc, got: ", m[1])
    
    two = model_names[-2]
    m = re.findall(r'[a-z]+', two)
    assert m[1] == "gan", print("wanted: gan, got: ",m[1])
    
    three = model_names[-1]
    m = re.findall(r'[a-z]+', three)
    assert m[1] == "gen", print("wanted: gen, got: ", m[1])
    
    img_input = Input(shape=(32,32,3))
    discriminator, disc_out = get_discriminator(img_input)
    discriminator.load_weights(modelpath + one)
    discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

    discriminator.trainable = False

    noise_input = Input(shape=(100,))
    generator, gen_out = get_generator(noise_input)
    generator.load_weights(modelpath + three)

    gan_input = Input(shape=(100,))    
    x = generator(gan_input)
    gan_out = discriminator(x)
    gan = Model(gan_input, gan_out)
    gan.summary()

    gan.load_weights(modelpath + two)
    gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')
    
    m = re.findall(r'[0-9]+', one)
    starting_epoch = int(m[1])
else:
    img_input = Input(shape=(32,32,3))
    discriminator, disc_out = get_discriminator(img_input)
    discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

    discriminator.trainable = False

    noise_input = Input(shape=(100,))
    generator, gen_out = get_generator(noise_input)

    gan_input = Input(shape=(100,))    
    x = generator(gan_input)
    gan_out = discriminator(x)
    gan = Model(gan_input, gan_out)
    gan.summary()

    gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       262272    
_________________________________________________________________
batch_normalization_2

In [10]:
BATCH_SIZE = 50

## MODIFIDED TO TAKE IMAGES FROM FOLDER########
img_names = os.listdir(foldernamepath)

# # Get training images

# ## Find the 10 classes with most data in our training set to train them 
# ## separately
# freq = [0] * 100

# for img in img_names:
#     m = re.findall(r'[0-9]+', img)
#     classNum = int(m[0])
#     freq[classNum] = freq[classNum] + 1

# sortedFreq = [0] * 100
# for i in range(len(freq)):
#     sortedFreq[i] = (i, freq[i])

# sortedFreq = sorted(sortedFreq, key = lambda x : x[1], reverse = True)

# # print(sortedFreq[:10])

# mostFreq = []
# for dataClass in sortedFreq[:10]:
#     mostFreq.append(dataClass[0])

# # print(mostFreq)

# ### Largest 10 Classes ###
# Class 28 with Frequency 446
# Class 54 with Frequency 324
# Class 3 with Frequency 260
# Class 5 with Frequency 194

# Class 55 with Frequency 162
# Class 35 with Frequency 156
# Class 7 with Frequency 152
# Class 30 with Frequency 150
# Class 16 with Frequency 142
# Class 11 with Frequency 138

classImg = []

# training on one class at a time
for img in img_names:
    m = re.findall(r'[0-9]+', img)
    if int(m[0]) == classToTrain:
        classImg.append(img)

img_names = classImg
print("Training on # of Images: ", len(img_names))

# training on the img files in img_names
random.shuffle(img_names)
x = []
for img_name in img_names:
    if img_name is not None:
        im = np.asarray(cv2.imread(foldernamepath + "/" + img_name, cv2.IMREAD_COLOR))
        if im.shape[0] == 32 and im.shape[1] == 32:
            x.append(im)

X_train = np.asarray(x)  

print ("Training shape: {}".format(X_train.shape))

# Normalize data
X_train = (X_train - 127.5) / 127.5
 
num_batches = int(X_train.shape[0]/BATCH_SIZE)

Training on # of Images:  138
Training shape: (138, 32, 32, 3)


In [0]:
if runOnlyOneModel:
    for i in range(50):
        show_imgs("class_" + str(classToTrain) + "_singleImage_" + str(i) + "_fromModel_" + str(starting_epoch))
else:
    N_EPOCHS = 3000
    for epoch in range(N_EPOCHS):

      cum_d_loss = 0.
      cum_g_loss = 0.

      for batch_idx in range(num_batches):
        # Get the next set of real images to be used in this iteration
        images = X_train[batch_idx*BATCH_SIZE : (batch_idx+1)*BATCH_SIZE]

        noise_data = generate_noise(BATCH_SIZE, 100)
        generated_images = generator.predict(noise_data)

        # Train on soft labels (add noise to labels as well)
        noise_prop = 0.05 # Randomly flip 5% of labels

        # Prepare labels for real data
        true_labels = np.zeros((BATCH_SIZE, 1)) + np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
        flipped_idx = np.random.choice(np.arange(len(true_labels)), size=int(noise_prop*len(true_labels)))
        true_labels[flipped_idx] = 1 - true_labels[flipped_idx]

        # Train discriminator on real data
        d_loss_true = discriminator.train_on_batch(images, true_labels)

        # Prepare labels for generated data
        gene_labels = np.ones((BATCH_SIZE, 1)) - np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
        flipped_idx = np.random.choice(np.arange(len(gene_labels)), size=int(noise_prop*len(gene_labels)))
        gene_labels[flipped_idx] = 1 - gene_labels[flipped_idx]

        # Train discriminator on generated data
        d_loss_gene = discriminator.train_on_batch(generated_images, gene_labels)

        d_loss = 0.5 * np.add(d_loss_true, d_loss_gene)
        cum_d_loss += d_loss

        # Train generator
        noise_data = generate_noise(BATCH_SIZE, 100)
        g_loss = gan.train_on_batch(noise_data, np.zeros((BATCH_SIZE, 1)))
        cum_g_loss += g_loss

      print('  Epoch: {}, Generator Loss: {}, Discriminator Loss: {}'.format(epoch+1+starting_epoch, cum_g_loss/num_batches, cum_d_loss/num_batches))

      if epoch % 50 == 0 or epoch <= 50:
          show_imgs("epoch" + str(epoch + starting_epoch))

      if epoch % 500 == 0 and epoch != 0:
        generator.save_weights(modelpath + "class_" + str(classToTrain) + "_{}_gen.h5".format(epoch + starting_epoch))
        discriminator.save_weights(modelpath +  "class_" + str(classToTrain) + "_{}_disc.h5".format(epoch + starting_epoch))
        gan.save_weights(modelpath +  "class_" + str(classToTrain) + "_{}_gan.hdf5".format(epoch + starting_epoch))
